In [1]:
from google_play_scraper import Sort, reviews
import pandas as pd

# List of bank apps with package names (update if incorrect)
apps = {
    "Commercial Bank of Ethiopia": "com.combanketh.mobilebanking",
    "Bank of Abyssinia": "com.boa.boaMobileBanking",
    "Dashen Bank": "com.cr2.amolelight"
}


# Desired number of reviews per app
N_REVIEWS = 400

# Master DataFrame to hold all reviews
all_reviews = pd.DataFrame()

for bank_name, app_id in apps.items():
    print(f"📦 Fetching reviews for {bank_name}...")
    
    reviews_list, _ = reviews(
        app_id,
        lang='en',
        country='us',
        sort=Sort.NEWEST,
        count=N_REVIEWS,
        filter_score_with=None  # Pull all ratings, not just 1-star, etc.
    )

    df = pd.DataFrame(reviews_list)
    df['bank'] = bank_name
    all_reviews = pd.concat([all_reviews, df], ignore_index=True)

# Save the scraped data
output_path = "bank_reviews.csv"
all_reviews.to_csv(output_path, index=False)
print(f"\n✅ Scraping complete. Data saved to: {output_path}")


📦 Fetching reviews for Commercial Bank of Ethiopia...
📦 Fetching reviews for Bank of Abyssinia...
📦 Fetching reviews for Dashen Bank...

✅ Scraping complete. Data saved to: bank_reviews.csv


C:\Users\Amenzz\AppData\Local\Temp\ipykernel_17540\3531052261.py:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_reviews = pd.concat([all_reviews, df], ignore_index=True)


In [2]:
import pandas as pd
from pathlib import Path

# 1️⃣  Load the raw scrape
raw_path = Path("bank_reviews.csv")
df = pd.read_csv(raw_path)

# 2️⃣  Basic cleaning
# -------------------------------------------------
# a) Drop exact duplicates based on the reviewId (if present)
if "reviewId" in df.columns:
    df = df.drop_duplicates(subset="reviewId")

# b) Drop rows with missing review text or rating
df = df.dropna(subset=["content", "score"])

# 3️⃣  Normalise the date
# -------------------------------------------------
# 'at' column from google-play-scraper is a full timestamp
df["date"] = pd.to_datetime(df["at"], errors="coerce").dt.date  # keep only YYYY-MM-DD
df = df.dropna(subset=["date"])  # remove rows we couldn’t parse

# 4️⃣  Keep / rename the requested columns
# -------------------------------------------------
df_clean = df.rename(columns={
    "content": "review",
    "score": "rating"
})[["review", "rating", "date", "bank"]]

df_clean["source"] = "Google Play"

# 5️⃣  Save the cleaned file
# -------------------------------------------------
clean_path = Path("bank_reviews_clean.csv")
df_clean.to_csv(clean_path, index=False)

print("✅ Clean file saved to:", clean_path)
print(df_clean.head())


✅ Clean file saved to: bank_reviews_clean.csv
                                              review  rating        date  \
0                         So bad now and hard to use       5  2025-06-09   
1  it is so amazing app. but, it is better to upd...       5  2025-06-09   
2                                         v.good app       4  2025-06-09   
3                                      very good app       1  2025-06-09   
4           Very amazing app indeed. I'm enjoying it       5  2025-06-08   

                          bank       source  
0  Commercial Bank of Ethiopia  Google Play  
1  Commercial Bank of Ethiopia  Google Play  
2  Commercial Bank of Ethiopia  Google Play  
3  Commercial Bank of Ethiopia  Google Play  
4  Commercial Bank of Ethiopia  Google Play  


In [3]:
import pandas as pd

# Load the cleaned dataset
df = pd.read_csv("bank_reviews_clean.csv")

# Show unique bank names
print("🏦 Banks found in dataset:", df['bank'].unique())


🏦 Banks found in dataset: ['Commercial Bank of Ethiopia' 'Bank of Abyssinia' 'Dashen Bank']


In [5]:
import pandas as pd

# Load your cleaned dataset
df = pd.read_csv("bank_reviews.csv")  # Replace with your actual CSV file path

# 1. 🔢 Number of reviews per bank
print("🔍 Number of Reviews Per Bank:")
print(df['bank'].value_counts())

# 2. 📉 Percentage of missing data per column
print("\n🧪 Percentage of Missing Data Per Column:")
missing_percent = df.isnull().mean() * 100
print(missing_percent.round(2))


🔍 Number of Reviews Per Bank:
bank
Commercial Bank of Ethiopia    400
Bank of Abyssinia              400
Dashen Bank                    400
Name: count, dtype: int64

🧪 Percentage of Missing Data Per Column:
reviewId                 0.00
userName                 0.00
userImage                0.00
content                  0.00
score                    0.00
thumbsUpCount            0.00
reviewCreatedVersion    25.33
at                       0.00
replyContent            99.92
repliedAt               99.92
appVersion              25.33
bank                     0.00
dtype: float64


# Task 2 Sentiment and Thematic Analysis

In [10]:
import pandas as pd
from transformers import pipeline

# Load the cleaned reviews
df = pd.read_csv("bank_reviews_clean.csv")

# Load the sentiment-analysis pipeline
sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

# Function to apply sentiment analysis
def analyze_sentiment(text):
    result = sentiment_analyzer(text[:512])[0]  # truncate long reviews
    label = result['label']
    score = result['score']
    return label, score

# Apply to each review
df[['sentiment_label', 'sentiment_score']] = df['review'].apply(lambda x: pd.Series(analyze_sentiment(x)))

# View sample results
print(df[['review', 'sentiment_label', 'sentiment_score']].head())

# Optionally, save to new CSV
df.to_csv("bank_reviews_with_sentiment.csv", index=False)


Device set to use cpu


                                              review sentiment_label  \
0                         So bad now and hard to use        NEGATIVE   
1  it is so amazing app. but, it is better to upd...        POSITIVE   
2                                         v.good app        POSITIVE   
3                                      very good app        POSITIVE   
4           Very amazing app indeed. I'm enjoying it        POSITIVE   

   sentiment_score  
0         0.999806  
1         0.949643  
2         0.995270  
3         0.999868  
4         0.999882  


In [12]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Load the cleaned reviews
df = pd.read_csv("bank_reviews_clean.csv")

# Initialize VADER
analyzer = SentimentIntensityAnalyzer()

# Define VADER sentiment scoring
def vader_sentiment(text):
    vs = analyzer.polarity_scores(text)
    compound = vs['compound']
    if compound >= 0.05:
        label = 'POSITIVE'
    elif compound <= -0.05:
        label = 'NEGATIVE'
    else:
        label = 'NEUTRAL'
    return label, compound

# Apply VADER sentiment analysis
df[['vader_label', 'vader_score']] = df['review'].apply(lambda x: pd.Series(vader_sentiment(x)))

# Display a few results
print(df[['review', 'vader_label', 'vader_score']].head())

# Optional: Save with VADER sentiment
df.to_csv("bank_reviews_with_vader.csv", index=False)
#compare DistilBERT and VADER outputs side-by-side
# print(df[['review', 'sentiment_label', 'sentiment_score', 'vader_label', 'vader_score']].head())


                                              review vader_label  vader_score
0                         So bad now and hard to use    NEGATIVE      -0.6361
1  it is so amazing app. but, it is better to upd...    POSITIVE       0.9049
2                                         v.good app     NEUTRAL       0.0000
3                                      very good app    POSITIVE       0.4927
4           Very amazing app indeed. I'm enjoying it    POSITIVE       0.8173


In [13]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# 1. Load cleaned reviews
df = pd.read_csv("bank_reviews_clean.csv")

# 2. Initialize VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# 3. Define sentiment function
def vader_sentiment(text):
    vs = analyzer.polarity_scores(text)
    compound = vs['compound']
    if compound >= 0.05:
        label = 'POSITIVE'
    elif compound <= -0.05:
        label = 'NEGATIVE'
    else:
        label = 'NEUTRAL'
    return label, compound

# 4. Apply sentiment analysis to each review
df[['vader_label', 'vader_score']] = df['review'].apply(lambda x: pd.Series(vader_sentiment(x)))

# 5. Save to file with sentiment results (optional)
df.to_csv("bank_reviews_with_vader.csv", index=False)

# 6. Aggregate by bank and rating
agg_sentiment = df.groupby(['bank', 'rating']).agg(
    mean_vader_score=('vader_score', 'mean'),
    review_count=('review', 'count')
).reset_index()

# 7. Show results
print("\n✅ Aggregated Sentiment by Bank and Rating:\n")
print(agg_sentiment)

# 8. Save aggregation to CSV (optional)
agg_sentiment.to_csv("bank_reviews_aggregated_sentiment.csv", index=False)



✅ Aggregated Sentiment by Bank and Rating:

                           bank  rating  mean_vader_score  review_count
0             Bank of Abyssinia       1         -0.157021           164
1             Bank of Abyssinia       2          0.132891            11
2             Bank of Abyssinia       3          0.192516            31
3             Bank of Abyssinia       4          0.333017            18
4             Bank of Abyssinia       5          0.348110           176
5   Commercial Bank of Ethiopia       1          0.003700            49
6   Commercial Bank of Ethiopia       2          0.053775            16
7   Commercial Bank of Ethiopia       3          0.174140            20
8   Commercial Bank of Ethiopia       4          0.333955            40
9   Commercial Bank of Ethiopia       5          0.422128           275
10                  Dashen Bank       1         -0.084621            61
11                  Dashen Bank       2          0.023394            16
12                 

# Thematic Analysis

In [14]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
from collections import Counter

# Load cleaned reviews
df = pd.read_csv("bank_reviews_clean.csv")

# Filter short reviews
df = df[df['review'].str.len() > 20]

# -------------------------------
# 1️⃣ TF-IDF Keyword Extraction
# -------------------------------
vectorizer = TfidfVectorizer(
    stop_words='english',
    ngram_range=(1, 2),     # unigrams and bigrams
    max_df=0.9,
    min_df=5                # appears in at least 5 reviews
)

tfidf_matrix = vectorizer.fit_transform(df['review'])
feature_names = vectorizer.get_feature_names_out()

def get_top_keywords(matrix, features, top_n=30):
    sums = matrix.sum(axis=0).A1
    data = [(word, sums[idx]) for idx, word in enumerate(features)]
    return sorted(data, key=lambda x: x[1], reverse=True)[:top_n]

top_keywords = get_top_keywords(tfidf_matrix, feature_names, top_n=30)

print("\n✅ Top TF-IDF Keywords:")
for word, score in top_keywords:
    print(f"{word:<20} {score:.2f}")

# -------------------------------
# 2️⃣ Manual Rule-Based Clustering
# -------------------------------
theme_map = {
    "login_issues": ["login", "sign in", "authentication", "credentials"],
    "performance": ["slow", "crash", "freeze", "lag", "unresponsive"],
    "customer_service": ["support", "help", "customer care", "agent", "response"],
    "transactions": ["transfer", "deposit", "withdraw", "balance", "payment"],
    "usability": ["interface", "design", "navigation", "user-friendly", "easy to use"]
}

# Create reverse mapping
keyword_to_theme = {}
for theme, words in theme_map.items():
    for w in words:
        keyword_to_theme[w.lower()] = theme

# Group keywords into themes
theme_keywords = {theme: [] for theme in theme_map}
for word, score in top_keywords:
    for key in keyword_to_theme:
        if key in word:
            theme_keywords[keyword_to_theme[key]].append(word)

# Show clustered keywords
print("\n✅ Grouped Keywords by Theme:")
for theme, keywords in theme_keywords.items():
    print(f"{theme}: {set(keywords)}")

# -------------------------------
# 3️⃣ (Optional) spaCy Phrase Extraction
# -------------------------------
nlp = spacy.load("en_core_web_sm")

def extract_noun_phrases(text):
    doc = nlp(text)
    return [chunk.text.lower() for chunk in doc.noun_chunks if len(chunk.text.split()) <= 3]

# Apply to all reviews
all_phrases = df['review'].apply(extract_noun_phrases)
flat_phrases = [phrase for sublist in all_phrases for phrase in sublist]
phrase_counts = Counter(flat_phrases).most_common(30)

print("\n✅ Top Noun Phrases from spaCy:")
for phrase, count in phrase_counts:
    print(f"{phrase:<30} {count}")



✅ Top TF-IDF Keywords:
app                  60.57
good                 24.10
bank                 22.74
work                 19.06
use                  18.86
banking              16.23
application          15.90
best                 15.23
mobile               13.82
like                 13.47
working              11.99
time                 11.70
mobile banking       11.61
update               10.88
make                 10.81
ነው                   10.51
fix                  10.45
doesn                10.33
don                  10.22
worst                10.05
cbe                  10.03
easy                 9.61
nice                 9.22
account              8.82
screenshot           8.80
better               8.54
money                8.42
good app             8.39
need                 8.23
slow                 8.12

✅ Grouped Keywords by Theme:
login_issues: set()
performance: {'slow'}
customer_service: set()
transactions: set()
usability: set()

✅ Top Noun Phrases from spaCy:
it        

In [15]:
import pandas as pd
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter, defaultdict

# Load data
df = pd.read_csv("bank_reviews_clean.csv")
df = df[df['review'].str.len() > 20]  # filter out very short reviews

# Load spaCy
nlp = spacy.load("en_core_web_sm")

# -------------------------------
# STEP 1: Extract TF-IDF Keywords
# -------------------------------
vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), max_df=0.9, min_df=3)
tfidf_matrix = vectorizer.fit_transform(df['review'])
feature_names = vectorizer.get_feature_names_out()

# Get top N keywords
def get_top_keywords(tfidf_matrix, feature_names, top_n=50):
    sums = tfidf_matrix.sum(axis=0).A1
    keywords = [(feature_names[i], sums[i]) for i in range(len(sums))]
    return sorted(keywords, key=lambda x: x[1], reverse=True)[:top_n]

# -------------------------------
# STEP 2: Extract Noun Phrases
# -------------------------------
def extract_noun_phrases(text):
    doc = nlp(text)
    return [chunk.text.lower().strip() for chunk in doc.noun_chunks if len(chunk.text.split()) <= 3]

df["noun_phrases"] = df["review"].apply(extract_noun_phrases)

# Flatten and count phrases
flat_phrases = [phrase for phrases in df["noun_phrases"] for phrase in phrases]
top_phrases = Counter(flat_phrases).most_common(50)

# -------------------------------
# STEP 3: Define Theme Mapping Logic
# -------------------------------
theme_definitions = {
    "Account Access Issues": ["login", "sign in", "authentication", "password", "otp", "access"],
    "Transaction Performance": ["transfer", "deposit", "payment", "balance", "slow", "delay", "failed", "transaction"],
    "User Interface & Experience": ["interface", "design", "navigation", "user-friendly", "app", "crash", "freeze", "slow", "responsive"],
    "Customer Support": ["support", "help", "agent", "service", "response", "call", "email"],
    "Feature Requests": ["feature", "add", "option", "would like", "missing", "need", "request"]
}

def match_theme(keyword):
    keyword_lower = keyword.lower()
    for theme, terms in theme_definitions.items():
        if any(term in keyword_lower for term in terms):
            return theme
    return "Other"

# -------------------------------
# STEP 4: Group by Bank
# -------------------------------
bank_theme_keywords = defaultdict(lambda: defaultdict(int))

for bank in df["bank"].unique():
    bank_df = df[df["bank"] == bank]

    # Extract TF-IDF per bank
    bank_vector = vectorizer.fit_transform(bank_df["review"])
    bank_keywords = get_top_keywords(bank_vector, vectorizer.get_feature_names_out(), top_n=50)

    for keyword, score in bank_keywords:
        theme = match_theme(keyword)
        bank_theme_keywords[bank][theme] += 1

# -------------------------------
# STEP 5: Display Final Themes Per Bank
# -------------------------------
for bank, themes in bank_theme_keywords.items():
    print(f"\n📊 Themes for {bank}:")
    sorted_themes = sorted(themes.items(), key=lambda x: x[1], reverse=True)
    for theme, count in sorted_themes:
        print(f" - {theme}: {count} keywords")



📊 Themes for Commercial Bank of Ethiopia:
 - Other: 41 keywords
 - User Interface & Experience: 5 keywords
 - Transaction Performance: 2 keywords
 - Feature Requests: 1 keywords
 - Account Access Issues: 1 keywords

📊 Themes for Bank of Abyssinia:
 - Other: 41 keywords
 - User Interface & Experience: 6 keywords
 - Feature Requests: 2 keywords
 - Transaction Performance: 1 keywords

📊 Themes for Dashen Bank:
 - Other: 38 keywords
 - User Interface & Experience: 4 keywords
 - Feature Requests: 3 keywords
 - Account Access Issues: 2 keywords
 - Transaction Performance: 2 keywords
 - Customer Support: 1 keywords


In [17]:
import pandas as pd
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
from textblob import TextBlob  # For simple sentiment analysis

# Load data
df = pd.read_csv("bank_reviews_clean.csv")
df = df[df['review'].str.len() > 20].copy()  # Filter very short reviews

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# --- Preprocessing function ---
def preprocess_text(text):
    doc = nlp(text.lower())
    tokens = [
        token.lemma_ for token in doc
        if not token.is_stop and not token.is_punct and token.is_alpha
    ]
    return " ".join(tokens)

df["cleaned_review"] = df["review"].apply(preprocess_text)

# --- Sentiment Analysis function using TextBlob ---
def analyze_sentiment(text):
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity
    # Simple labeling logic
    if polarity > 0.1:
        label = "Positive"
    elif polarity < -0.1:
        label = "Negative"
    else:
        label = "Neutral"
    return label, polarity

df[["sentiment_label", "sentiment_score"]] = df["cleaned_review"].apply(
    lambda x: pd.Series(analyze_sentiment(x))
)

# --- Theme definitions ---
theme_definitions = {
    "Account Access Issues": ["login", "sign in", "authentication", "password", "otp", "access"],
    "Transaction Performance": ["transfer", "deposit", "payment", "balance", "slow", "delay", "failed", "transaction"],
    "User Interface & Experience": ["interface", "design", "navigation", "user-friendly", "app", "crash", "freeze", "slow", "responsive"],
    "Customer Support": ["support", "help", "agent", "service", "response", "call", "email"],
    "Feature Requests": ["feature", "add", "option", "would like", "missing", "need", "request"]
}

# --- Theme matching function ---
def match_themes(text):
    matched = set()
    text_lower = text.lower()
    for theme, terms in theme_definitions.items():
        if any(term in text_lower for term in terms):
            matched.add(theme)
    if not matched:
        matched.add("Other")
    return list(matched)

df["identified_themes"] = df["cleaned_review"].apply(match_themes)

# --- Save to CSV ---
df = df.reset_index(drop=True)  # ensure clean numeric index
df['review_id'] = df.index + 1  # start IDs at 1 (optional)

output_columns = ["review_id", "review", "sentiment_label", "sentiment_score", "identified_themes"]

df.to_csv("processed_bank_reviews.csv", columns=output_columns, index=False)


print("Pipeline completed, results saved to processed_bank_reviews.csv")


Pipeline completed, results saved to processed_bank_reviews.csv


In [18]:
import pandas as pd
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from sklearn.cluster import KMeans

# Load spaCy
nlp = spacy.load("en_core_web_sm")

# Load data
df = pd.read_csv("bank_reviews_clean.csv")
df = df[df['review'].str.len() > 20]

# Function to extract noun phrases using spaCy (max 3 words)
def extract_noun_phrases(text):
    doc = nlp(text.lower())
    return [chunk.text.strip() for chunk in doc.noun_chunks if len(chunk.text.split()) <= 3]

# Prepare storage for results
bank_keywords = {}

for bank in df['bank'].unique():
    bank_df = df[df['bank'] == bank]
    
    # --- TF-IDF keywords ---
    vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=0.85, min_df=3)
    tfidf_matrix = vectorizer.fit_transform(bank_df['review'])
    feature_names = vectorizer.get_feature_names_out()
    
    sums = tfidf_matrix.sum(axis=0).A1
    keywords_scores = list(zip(feature_names, sums))
    keywords_scores = sorted(keywords_scores, key=lambda x: x[1], reverse=True)[:100]  # top 100
    
    # --- spaCy noun phrases ---
    noun_phrases = []
    for text in bank_df['review']:
        noun_phrases.extend(extract_noun_phrases(text))
    
    # Count noun phrases and take top 50
    np_counts = Counter(noun_phrases).most_common(50)
    
    # Combine keywords (TF-IDF + noun phrases) with scores/counts
    combined_keywords = {}
    for k, score in keywords_scores:
        combined_keywords[k] = combined_keywords.get(k, 0) + score
    for np, count in np_counts:
        combined_keywords[np] = combined_keywords.get(np, 0) + count * 0.5  # weigh noun phrases lower
    
    # Get combined top keywords for clustering
    top_keywords = sorted(combined_keywords.items(), key=lambda x: x[1], reverse=True)[:100]
    bank_keywords[bank] = [kw for kw, score in top_keywords]


In [19]:
import numpy as np

def get_phrase_vector(phrase):
    # average word vectors for multi-word phrases
    doc = nlp(phrase)
    if len(doc) == 0:
        return np.zeros(nlp.vocab.vectors_length)
    return doc.vector

# For each bank, cluster keywords into 3–5 themes
bank_clusters = {}

for bank, keywords in bank_keywords.items():
    vectors = np.array([get_phrase_vector(k) for k in keywords])
    
    # Choose number of clusters dynamically or fixed (e.g., 4)
    n_clusters = 4 if len(keywords) >= 4 else len(keywords)
    
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    labels = kmeans.fit_predict(vectors)
    
    clusters = defaultdict(list)
    for label, keyword in zip(labels, keywords):
        clusters[label].append(keyword)
    
    bank_clusters[bank] = clusters


In [20]:
for bank, clusters in bank_clusters.items():
    print(f"\nThemes for {bank}:")
    for cluster_id, keywords in clusters.items():
        print(f" Theme {cluster_id + 1}:")
        print("   ", ", ".join(keywords[:10]))  # show top 10 keywords in cluster



Themes for Commercial Bank of Ethiopia:
 Theme 1:
    it, i, you, screenshot, bank, application, banking, use, update, me
 Theme 3:
    app, cbe, this app, the app, mobile banking, good app, mobile, time, fix, ነው
 Theme 4:
    good, like, best, nice, great, that, easy, amazing, reliable, really
 Theme 2:
    make, work, using, account, need, thank, send, try, working, feature

Themes for Bank of Abyssinia:
 Theme 4:
    it, i, you, me, they, transactions, apps, works, crashes, times
 Theme 2:
    app, bank, boa, money, work, mobile banking, banking, use, working, doesn
 Theme 3:
    this app, the app, this, that, what, like, the bank, my phone, this bank, dont
 Theme 1:
    worst, better, best, just, frequently, long

Themes for Dashen Bank:
 Theme 3:
    it, i, app, you, this app, bank, amole, the app, dashen bank, good app
 Theme 1:
    good, best, dashen, fast, nice, easy, slow, mobile, that, simple
 Theme 2:
    use, update, work, account, like, used, transfer, working, make, need

In [21]:
import pandas as pd
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter, defaultdict
from sklearn.cluster import KMeans
import numpy as np
from textblob import TextBlob

# Load spaCy model once
nlp = spacy.load("en_core_web_sm")

def load_and_filter_data(filepath, min_review_len=20):
    df = pd.read_csv(filepath)
    df = df[df['review'].str.len() > min_review_len].copy()
    df.reset_index(drop=True, inplace=True)
    return df

def extract_noun_phrases(text):
    doc = nlp(text.lower())
    return [chunk.text.strip() for chunk in doc.noun_chunks if len(chunk.text.split()) <= 3]

def compute_sentiment(text):
    # TextBlob polarity: -1 (negative) to +1 (positive)
    blob = TextBlob(text)
    return blob.sentiment.polarity

def get_tfidf_keywords(corpus, top_n=100):
    vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=0.85, min_df=3)
    tfidf_matrix = vectorizer.fit_transform(corpus)
    feature_names = vectorizer.get_feature_names_out()
    sums = tfidf_matrix.sum(axis=0).A1
    keywords_scores = list(zip(feature_names, sums))
    return sorted(keywords_scores, key=lambda x: x[1], reverse=True)[:top_n]

def combine_keywords(tfidf_keywords, noun_phrases_counts, weight_np=0.5, top_n=100):
    combined = defaultdict(float)
    for kw, score in tfidf_keywords:
        combined[kw] += score
    for np, count in noun_phrases_counts:
        combined[np] += count * weight_np
    combined_sorted = sorted(combined.items(), key=lambda x: x[1], reverse=True)[:top_n]
    return [kw for kw, _ in combined_sorted]

def phrase_vector(phrase):
    doc = nlp(phrase)
    if len(doc) == 0:
        return np.zeros(nlp.vocab.vectors_length)
    return doc.vector

def cluster_keywords(keywords, n_clusters=4):
    vectors = np.array([phrase_vector(k) for k in keywords])
    n_clusters = min(n_clusters, len(keywords))
    if n_clusters == 0:
        return {}
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    labels = kmeans.fit_predict(vectors)
    clusters = defaultdict(list)
    for label, keyword in zip(labels, keywords):
        clusters[label].append(keyword)
    return clusters

def assign_themes_to_reviews(df, theme_keywords_per_bank):
    # For each review, assign themes that appear in its text
    themes_list = []
    for _, row in df.iterrows():
        review_text = row['review'].lower()
        bank = row['bank']
        themes = []
        for theme_id, keywords in theme_keywords_per_bank.get(bank, {}).items():
            if any(k in review_text for k in keywords):
                themes.append(f"Theme {theme_id+1}")
        themes_list.append(", ".join(themes) if themes else "Other")
    df['identified_themes'] = themes_list
    return df

def pipeline(filepath, n_themes=4):
    df = load_and_filter_data(filepath)
    
    # Sentiment scoring
    df['sentiment_score'] = df['review'].apply(compute_sentiment)
    df['sentiment_label'] = df['sentiment_score'].apply(lambda x: 'positive' if x > 0.1 else ('negative' if x < -0.1 else 'neutral'))
    
    bank_theme_keywords = {}
    
    for bank in df['bank'].unique():
        bank_df = df[df['bank'] == bank]
        
        # Extract TF-IDF keywords
        tfidf_keywords = get_tfidf_keywords(bank_df['review'], top_n=100)
        
        # Extract noun phrases counts
        all_noun_phrases = []
        for text in bank_df['review']:
            all_noun_phrases.extend(extract_noun_phrases(text))
        np_counts = Counter(all_noun_phrases).most_common(50)
        
        # Combine keywords
        combined_keywords = combine_keywords(tfidf_keywords, np_counts, weight_np=0.5, top_n=100)
        
        # Cluster combined keywords into themes
        clusters = cluster_keywords(combined_keywords, n_clusters=n_themes)
        bank_theme_keywords[bank] = clusters
        
    # Assign themes per review
    df = assign_themes_to_reviews(df, bank_theme_keywords)
    
    # Show themes summary per bank
    for bank, clusters in bank_theme_keywords.items():
        print(f"\n🎯 Themes for {bank}:")
        for cid, keywords in clusters.items():
            print(f" Theme {cid+1}: {', '.join(keywords[:10])}")
    
    # Save results to CSV
    output_cols = ['review', 'bank', 'sentiment_label', 'sentiment_score', 'identified_themes']
    df.to_csv("processed_bank_reviews.csv", columns=output_cols, index=False)
    print("\n✅ Pipeline completed and saved to processed_bank_reviews.csv")
    
    return df, bank_theme_keywords

# Run pipeline
df_result, themes_per_bank = pipeline("bank_reviews_clean.csv", n_themes=4)



🎯 Themes for Commercial Bank of Ethiopia:
 Theme 1: it, i, you, screenshot, bank, application, banking, use, update, me
 Theme 3: app, cbe, this app, the app, mobile banking, good app, mobile, time, fix, ነው
 Theme 4: good, like, best, nice, great, that, easy, amazing, reliable, really
 Theme 2: make, work, using, account, need, thank, send, try, working, feature

🎯 Themes for Bank of Abyssinia:
 Theme 4: it, i, you, me, they, transactions, apps, works, crashes, times
 Theme 2: app, bank, boa, money, work, mobile banking, banking, use, working, doesn
 Theme 3: this app, the app, this, that, what, like, the bank, my phone, this bank, dont
 Theme 1: worst, better, best, just, frequently, long

🎯 Themes for Dashen Bank:
 Theme 3: it, i, app, you, this app, bank, amole, the app, dashen bank, good app
 Theme 1: good, best, dashen, fast, nice, easy, slow, mobile, that, simple
 Theme 2: use, update, work, account, like, used, transfer, working, make, need
 Theme 4: what, which

✅ Pipeline com

# Task 3 Store Cleaned Data in Oracle


In [ ]:
import cx_Oracle
import pandas as pd


# Oracle connection info
username = "***********"
password = "*********"
dsn = "localhost/XEPDB1"  # Adjust for your Oracle XE service name and host

connection = cx_Oracle.connect(username, password, dsn)
cursor = connection.cursor()

# Step 1: Insert unique banks into banks table
banks = df_result['bank'].unique()
bank_id_map = {}

for bank_name in banks:
    cursor.execute("INSERT INTO banks (bank_name) VALUES (:1) RETURNING bank_id INTO :2",
                   (bank_name, cx_Oracle.NUMBER))
    bank_id = cursor.getimplicitresults()[0][0]
    # If RETURNING doesn't work, fallback:
    # cursor.execute("SELECT bank_id FROM banks WHERE bank_name=:1", (bank_name,))
    # bank_id = cursor.fetchone()[0]
    bank_id_map[bank_name] = bank_id

connection.commit()

# Alternative approach to get IDs (if RETURNING is problematic):
# Insert bank names, then query all to build bank_id_map.

cursor.execute("SELECT bank_id, bank_name FROM banks")
for bank_id, bank_name in cursor.fetchall():
    bank_id_map[bank_name] = bank_id

# Step 2: Insert reviews
insert_sql = """
    INSERT INTO reviews (bank_id, review_text, sentiment_label, sentiment_score, identified_themes)
    VALUES (:bank_id, :review_text, :sentiment_label, :sentiment_score, :identified_themes)
"""

data_to_insert = []
for idx, row in df_result.iterrows():
    data_to_insert.append((
        bank_id_map[row['bank']],
        row['review'],
        row['sentiment_label'],
        float(row['sentiment_score']),
        row['identified_themes']
    ))

cursor.executemany(insert_sql, data_to_insert)
connection.commit()

print(f"Inserted {len(data_to_insert)} reviews into Oracle DB")

cursor.close()
connection.close()


#Task four

In [ ]:
# Top positive and negative themes
positive_reviews = df[df["sentiment_label"] == "positive"]
negative_reviews = df[df["sentiment_label"] == "negative"]

from collections import Counter

# Flatten themes for positive and negative
positive_themes = [theme for themes in positive_reviews["identified_themes"] for theme in themes.split(", ")]
negative_themes = [theme for themes in negative_reviews["identified_themes"] for theme in themes.split(", ")]

# Count them
top_positive = Counter(positive_themes).most_common(5)
top_negative = Counter(negative_themes).most_common(5)

print("✅ Top Drivers:")
for theme, count in top_positive:
    print(f"- {theme} ({count} mentions)")

print("\n❌ Top Pain Points:")
for theme, count in top_negative:
    print(f"- {theme} ({count} mentions)")


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Average sentiment by bank
sns.barplot(data=df, x="bank", y="sentiment_score")
plt.title("Average Sentiment Score by Bank")
plt.ylabel("Sentiment Score")
plt.xlabel("Bank")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
#visualizations
sns.countplot(data=df, x="sentiment_label", hue="bank")
plt.title("Sentiment Label Distribution by Bank")
plt.show()


In [ ]:
from wordcloud import WordCloud

positive_text = " ".join(positive_reviews["review"])
negative_text = " ".join(negative_reviews["review"])

WordCloud(background_color='white', width=800, height=400).generate(positive_text).to_image().show()
WordCloud(background_color='black', width=800, height=400).generate(negative_text).to_image().show()


In [ ]:
import pandas as pd

# Convert identified_themes to lists
df["themes_list"] = df["identified_themes"].apply(lambda x: x.split(", "))

from itertools import chain
theme_counts = pd.Series(list(chain(*df["themes_list"]))).value_counts().head(10)

theme_counts.plot(kind="bar", title="Top Themes in Reviews")
plt.ylabel("Number of Mentions")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
